# Criminal Soccer
**Team:** Michael Gangl, Sebastian Grünewald, Patrick Leitner

## Topic
The goal of the project is to load and process various datasets from the soccer area and visualize the results. The project is to be run and analyzed under the
Big Data aspect. For this purpose, relational and NoSQL databases were used, as well as a MapReduce algorithm. Furthermore, the architecture is built in a way that all services and metadata are hosted and is therefore multiuser capable.

## Question
With our diagrams we want to analyze if there are correlations between the foul statistics of the players and the crime statistics of their countries of origin.
Additionally, some other interesting and relevant graphs about the datasets will be shown.

## Architecture
[GitHub](https://github.com/petz1209/fh_bdi_soccer_project)

Azure: fhtw-bdinf-wi21b080-azuredatabase-mysql.mysql.database.azure.com
MySQL Database: `criminalsoccer`
MySQL Table: `nationalities`

MongoDB: mongodb://pt-n20.p4001.w3.cs.technikum-wien.at:4001
MongoDB Database: `criminalSoccer`
MongoDB Collections: `raw_players`, `players`, `countries`

![Architecture Diagram](imgs/Architecture_Diagram.jpg)

## Data Collection
The following scripts load the data from various sources and stores it into MongoDB
### Datasets
[Kaggle UEFA (.csv-files)](https://www.kaggle.com/datasets/azminetoushikwasi/ucl-202122-uefa-champions-league): This dataset contains all the player stats of UEFA Champions League season 2021-22.

[TransferMarkt-API](https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche): To complete the player stats, we used the Transfermarkt API to search for players by their name.

[Wikipedia National Crime Stats](https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate): Wikipedia provides a table with crime rates per 100.000 inhabitants.

### Technical Notice
**Make sure Docker is up and running!**
Due to historical problems with the hosted MongoDB instance from the FH, we implemented a backup strategy with Docker.

### Steps
1. Prepare local backup strategy for mongo with a docker container due to problems with the hosted mongo instance.
2. Download UEFA Dataset from kaggle and store the data in MongoDB.
3. Iterate over player names and scrape additional information from Transfermarkt.com API.
4. Transform the data with proper datatypes and cleanup combined dataset.
5. Scrape Wikipedia to get crime statistics per country and store it in MongoDB.

In [ ]:
import pandas as pd
import zipfile
import os
import requests

# Importing necessary packages for mongodb connectivity
try:
    from pymongo import MongoClient
    from pymongo.errors import ServerSelectionTimeoutError
except ImportError:
    !pip install pymongo[srv]
    from pymongo import MongoClient
    from pymongo.errors import ServerSelectionTimeoutError

# Importing config from config.py
from conifg import MONGO_HOST_REMOTE, MONGO_DB_REMOTE, MONGO_HOST_LOCAL, MONGO_DB_LOCAL

# Defining constants for kaggle files
UEFA_ZIP = "kaggle_players_zip.zip"
UEFA_UNZIPPED = "kaggle_files"
UEFA_FILES = ["key_stats.csv", "disciplinary.csv", "distributon.csv", "defending.csv"] # only some .csv-files are interesting for our purposes
UEFA_RAW_DATA = "raw_players"

# Defining constants for MongoDB connection
conn_str = MONGO_HOST_REMOTE
mongoDB = MONGO_DB_REMOTE

class MongoContext:
    """mongodb client context manager"""

    def __enter__(self):
        global conn_str, mongoDB
        try:
            # Connection to Mongo Server from FH-Technikum
            self.client = MongoClient(conn_str)
            self.client.server_info()
            #print("Connection successful to remote mongo host")
            return self.client
        # If connection is not possible, setting a local docker instance
        except ServerSelectionTimeoutError as err:
            print("Remote Error: " + str(err))
            os.system("docker pull mongo")
            os.system("docker run -d -p 27017:27017 mongo:latest")
            conn_str = MONGO_HOST_LOCAL
            mongoDB = MONGO_DB_LOCAL
            try:
                # Trying to connect to the local docker Mongo database
                self.client = MongoClient(conn_str)
                self.client.server_info()
                #print("Connection successful to local mongo host")
                return self.client
            except ServerSelectionTimeoutError as errLocal:
                print("Local Error: " + str(errLocal))

    def __exit__(self, exception_type, exception_value, exception_traceback):
        self.client.close()
        del self.client

def unpack_zip(src, dest):
    """takes files in zip folder from src and extracts them to dest"""
    with zipfile.ZipFile(src, 'r') as zip_ref:
        zip_ref.extractall(dest)

def csv_to_mongo(folder, files, map_key):
    """Fetching data from interesting files in csv folder"""
    # kill existing collection if it exists:
    with MongoContext() as client:
        db = client[mongoDB]
        collection = db[UEFA_RAW_DATA]
        collection.drop()

        for idx, file in enumerate(files):
            df = pd.read_csv(f"{folder}/{file}")
            data = df.to_dict(orient='records')
            if idx == 0:
                # Insert data into mongo
                collection.insert_many(data)
            else:
                for row in data:
                    # Insert data into mongo
                    query = {map_key:  row[map_key]}
                    new_values = {"$set": row}
                    collection.update_one(query, new_values)

def read_from_mongo():
    """reading from mongo database and printing the collection"""
    with MongoContext() as client:
        db = client[mongoDB]
        collection = db[UEFA_RAW_DATA]

        data = collection.find()
        for x in data:
            print("==========================================================================")
            print(x)

def collect_from_kaggle():

    # guard, in case data is already in database.
    with MongoContext() as client:
        db = client[mongoDB]
        if UEFA_RAW_DATA in db.list_collection_names():
            print(f"{UEFA_RAW_DATA} is already in database")
            return False

    unpack_zip(UEFA_ZIP, UEFA_UNZIPPED)
    csv_to_mongo(UEFA_UNZIPPED, UEFA_FILES, "player_name")
    read_from_mongo()

collect_from_kaggle()

### Transfermarkt Spider
<img src="imgs/Transfermarkt_logo.png" style="width:200px; height:auto"/>

In Order to collect insights on players nationality we are querying the Transfermarkt [website](https://transfermarkt.com).
We then parse the html response to collect `full_name`, `nationality`, `icon` and `market_value` from it.

In [ ]:
try:
    import parsel
except ImportError:
    !pip install parsel
    import parsel


def transfermarkt_spider(name):
    """queries transfermarkt.com and parses response table"""
    ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    header = {"user-agent": ua}
    result = None
    with requests.Session() as session:
        url = "https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche"
        req = session.get(url, params={"query": name}, headers=header) # API call
        response = parsel.Selector(req.text)
        try:
            # retrieve data from table
            row = response.xpath("//table[@class='items']/tbody/tr[1]")
            icon_url = row.xpath(".//table//img/@src").get()
            name = row.xpath(".//table//img/@title").get()
            national = row.xpath("./td[5]/img[1]/@alt").get()
            value = row.xpath("./td[6]/text()").get()
            result = dict(icon=icon_url, full_name=name, nationality=national, market_value=value)
        except:
            pass
    return result

# Collect from Transfermarkt.com
with MongoContext() as client:
    # MongoDB connection
    db = client[mongoDB]
    collection = db[UEFA_RAW_DATA]
    raw = collection.find()
    count = 0

    # add the complementary data to the mongo documents
    mongo_rows = collection.find({ "nationality": { "$exists":False }})
    for mongo_player in mongo_rows:
        print(mongo_player["player_name"])
        if not mongo_player.get("nationality"):
            name = mongo_player["player_name"]
            transfer_data = transfermarkt_spider(name)
            if transfer_data:
                print(transfer_data)
                collection.update_one({"player_name": name}, {"$set": transfer_data})

### Transform Data
In order to analyze the data we apply type conversions on selected attributes

In [ ]:
TYPE_CONVERSIONS = {"minutes_played": "int",
                    'match_played': "int", 'goals': "int", 'assists': "int", 'distance_covered': 'float',
                    'fouls_committed': "int", 'fouls_suffered': "int", 'red': "int", 'yellow': "int",
                    'cross_accuracy': "int", 'cross_attempted': "int", 'cross_complted': "int",
                    'freekicks_taken': "int", 'pass_accuracy': "float", 'pass_attempted': "int",
                    'pass_completed': "int",
                    'balls_recoverd': "int",
                    'clearance_attempted': "int",
                    't_lost': "int",
                    't_won': "int",
                    'tackles': "int"
                    }

def type_converter(item: dict, definitions) -> dict:
    """converts all values that are in a given type key"""
    new_item = dict()
    for k, v in item.items():
        if k in definitions:
            if definitions[k] == "int":
                try:
                    float(v)
                    v = int(v)
                except:
                    v = None
            elif definitions[k] == "float":
                try:
                    v = float(v)
                except:
                    v = None

        new_item[k] = v
    return new_item

def transform_raw_data():
    with MongoContext() as client:
        db = client[mongoDB]
        raw = db[UEFA_RAW_DATA]
        collection = db["players"]
        collection.drop()
        for doc in raw.find():
            cleaned_item = type_converter(doc, TYPE_CONVERSIONS)
            collection.insert_one(cleaned_item)

transform_raw_data()

### Fetch crime stats from Wikipedia:
Download crime stats (intentional homicides) for all countries in the world (0.0-10.0)


In [ ]:
def crime_from_wiki():
    """scrape from wikipedia and yield results"""
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate"
    with requests.Session() as session:
        req = session.get(url)
        response = parsel.Selector(req.text)
        table = response.xpath("//table[contains(@class,'static-row-numbers')]")
        body = table.xpath("./tbody//tr")
        for row in body:
            country = row.xpath("./td[1]//a/text()").get()
            if country:
                country = country.strip("*")
                country = country.strip()
                count_p_100k = float(row.xpath("./td[4]/text()").get())
                yield {"country":country, "count_p_100k":count_p_100k}

# Collect from Wikipedia
with MongoContext() as client:
    db= client[mongoDB]
    collection = db["countries"]
    collection.drop()
    for country in crime_from_wiki():
        collection.insert_one(country)

    for x in collection.find():
        print(x)

### Metadata in MongoDB
#### Collections
##### Players:
Example data entry from a soccer player:

```{'_id': ObjectId('643c1b35afd7d39c21e6dbf8'),'player_name': 'Courtois', 'club': 'Real Madrid', 'position': 'Goalkeeper', 'minutes_played': 1230, 'match_played': 13, 'goals': 0, 'assists': 0, 'distance_covered': 64.2, 'cross_accuracy': 0, 'cross_attempted': 0, 'cross_complted': 0, 'freekicks_taken': 27, 'pass_accuracy': 76.7, 'pass_attempted': 483, 'pass_completed': 365, 'serial': 447, 'full_name': 'Thibaut Courtois', 'icon': 'https://img.a.transfermarkt.technology/portrait/small/108390-1665067957.jpg?lm=1', 'market_value': '€60.00m', 'nationality': 'Belgium'}```

##### Countries:
Example data entry for the crime stats:

`{'_id': ObjectId('643c37c7afd7d39c21e6dfc0'), 'country': 'Afghanistan', 'count_p_100k': 6.7}`

## Data Analyzation with pandas

//TODO: @Patrick

### MapReduce
The goal of this MapReduce algorithm is, to count and sum up the total country appearances from all players.
Therefore, the documents need to loaded from MongoDB. After the MapReduce Job the result is stored into the MySQL database from Azure.

In [ ]:
%%file mapreduce.py # Writing this cell into a python file, so it is executable by cli
from mrjob.job import MRJob
import mysql.connector
import azure_credentials as az

# Establish connection to MySQL database hosted in Azure
cnx = mysql.connector.connect(user=az.user,
                              password=az.azure_mysql_password,
                              host="fhtw-bdinf-wi21b080-azuredatabase-mysql.mysql.database.azure.com",
                              port=3306,
                              database="criminalsoccer",
                              ssl_ca="DigiCertGlobalRootCA.crt.pem",
                              ssl_disabled=False)

cursor = cnx.cursor()

class CountNationalities(MRJob):
    global cnx, cursor
    def mapper(self, _, nationality):
        # Emit a key-value pair with the nationality as the key and 1 as the value
        yield (nationality, 1)

    def reducer(self, nationality, counts):
        # Sum the counts for each nationality
        total_count = sum(counts)

        # Store the results in the MySQL table
        insert_query = """INSERT INTO nationalities (country, appearances)
                           VALUES (%s, %s)
                           ON DUPLICATE KEY UPDATE country=country"""
        cursor.execute(insert_query, (nationality, total_count))
        cnx.commit()

if __name__ == '__main__':
    CountNationalities.run()
    cursor.close()
    cnx.close()

In [ ]:
import mysql.connector
from pymongo import MongoClient
import azure_credentials as az
import os

# Establish connection to MySQL database hosted in Azure
cnx = mysql.connector.connect(user=az.user,
                              password=az.azure_mysql_password,
                              host="fhtw-bdinf-wi21b080-azuredatabase-mysql.mysql.database.azure.com",
                              port=3306,
                              database="criminalsoccer",
                              ssl_ca="DigiCertGlobalRootCA.crt.pem",
                              ssl_disabled=False)
if cnx.is_connected():
    print("Successfully established connection!")
else:
    print("Failed connection!")
    exit()

cursor = cnx.cursor()

# Create table
create_table_query = """DROP TABLE IF EXISTS nationalities;
                        CREATE TABLE nationalities (
                        country VARCHAR(50) PRIMARY KEY,
                        appearances INT
                        )"""
cursor.execute(create_table_query, multi=True)

# Commit changes to the database
cnx.commit()


# Load all nationalities from MongoDB into a list
client = MongoClient('mongodb://pt-n20.p4001.w3.cs.technikum-wien.at:4001')
db = client['criminalSoccer']
collection = db['raw_players']

with open('nationalities.txt', 'w') as f:
    for document in collection.find():
        nationality = document.get('nationality', 'Unknown')
        f.write(str(nationality) + '\n')
print("Wrote nationalities to file")

os.system("python mapreduce.py nationalities.txt")

cursor.close()
cnx.close()

### Metadata in MySQL

In [ ]:
import mysql.connector
from pprint import pprint

# Establish a connection to the MySQL server
cnx = mysql.connector.connect(user=az.user,
                              password=az.azure_mysql_password,
                              host="fhtw-bdinf-wi21b080-azuredatabase-mysql.mysql.database.azure.com",
                              port=3306,
                              database="criminalsoccer",
                              ssl_ca="DigiCertGlobalRootCA.crt.pem",
                              ssl_disabled=False)

# Create a cursor object
mycursor = cnx.cursor()

# Execute a query to select data from the table
mycursor.execute("SELECT * FROM nationalities")

# Fetch all rows
rows = mycursor.fetchall()

# Pretty print the data
pprint(rows)


## Visualization

//TODO: @Michael

In [ ]:
import mysql.connector
import matplotlib.pyplot as plt
import azure_credentials as az

# Connect to MySQL database
db = mysql.connector.connect(user=az.user,
                              password=az.azure_mysql_password,
                              host="fhtw-bdinf-wi21b080-azuredatabase-mysql.mysql.database.azure.com",
                              port=3306,
                              database="criminalsoccer",
                              ssl_ca="DigiCertGlobalRootCA.crt.pem",
                              ssl_disabled=False)

# Retrieve data from table
cursor = db.cursor()
cursor.execute("SELECT country, appearances FROM nationalities")
results = cursor.fetchall()

# Prepare data for plotting
countries = [r[0] for r in results]
appearances = [r[1] for r in results]

# Create bar chart
fig, ax = plt.subplots(figsize=(15, 8))
ax.bar(countries, appearances)

# Customize chart
ax.set_title('Appearances by Country')
ax.set_xlabel('Country')
ax.set_ylabel('Number of Appearances')
plt.xticks(rotation=90)

# Show the chart
plt.show()
